<!--## Navigation: Overview + Detail-->

## Navegação: Visão geral e detalhes


<!--When panning and zooming, we directly adjust the "viewport" of a chart. The related navigation strategy of _overview + detail_ instead uses an overview display to show _all_ of the data, while supporting selections that pan and zoom a separate focus display.

Below we have two area charts showing a decade of price fluctuations for the S&amp;P 500 stock index. Initially both charts show the same data range. _Click and drag in the bottom overview chart to update the focus display and examine specific time spans._-->

Ao fazer a visão panorâmica e dar zoom, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada de _visão geral e detalhes_, em vez disso, usa uma exibição de visão geral para mostrar _todos_ os dados, oferecendo ainda o suporte a seleções que movimentam e ampliam uma exibição de foco separada.

Abaixo, temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, ambos os gráficos mostram o mesmo intervalo de dados. _Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos._

In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

<!--Unlike our earlier panning &amp; zooming case, here we don't want to bind a selection directly to the scales of a single interactive chart. Instead, we want to bind the selection to a scale domain in _another_ chart. To do so, we update the `x` encoding channel for our focus chart, setting the scale `domain` property to reference our `brush` selection. If no interval is defined (the selection is empty), Altair ignores the brush and uses the underlying data to determine the domain. When a brush interval is created, Altair instead uses that as the scale `domain` for the focus chart.-->

Ao contrário do nosso conjunto panorâmica e zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Para isso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para nosso gráfico principal, definindo a propriedade de escala `domain` (domínio) para referenciar nossa seleção de `brush` (Pincel). Se nenhum intervalo for definido (seleção vazia), o Altair ignora o pincel e usa os dados subjacentes para determinar o domínio. Quando um intervalo de pincel é criado, o Altair o usa como o `domain` da escala para o gráfico principal.

<!--## Details on Demand-->

## Detalhes sob demanda

<!--Once we spot points of interest within a visualization, we often want to know more about them. _Details-on-demand_ refers to interactively querying for more information about selected values. _Tooltips_ are one useful means of providing details on demand. However, tooltips typically only show information for one data point at a time. How might we show more?

The movie ratings scatterplot includes a number of potentially interesting outliers where the Rotten Tomatoes and IMDB ratings disagree. Let's create a plot that allows us to interactively select points and show their labels. To trigger the filter query on either the hover or click interaction, we will use the [Altair composition operator](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("or").

_Mouse over points in the scatter plot below to see a highlight and title label. Shift-click points to make annotations persistent and view multiple labels at once. Which movies are loved by Rotten Tomatoes critics, but not the general audience on IMDB (or vice versa)? See if you can find possible errors, where two different movies with the same name were accidentally combined!_-->

Após identificarmos pontos de interesse em uma visualização, por vezes queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à consulta interativa de mais informações sobre os valores selecionados. _Dicas de ferramentas_ são um meio útil de fornecer detalhes sob demanda. No entanto, as dicas de ferramentas normalmente mostram informações apenas para um ponto de dados por vez. Então, como podemos mostrar mais informações?

O gráfico de dispersão das classificações de filmes inclui uma série de valores discrepantes potencialmente interessantes onde as classificações do _Rotten Tomatoes_ e do _IMDB_ divergem. Vamos criar um gráfico que nos permita selecionar pontos de forma interativa e mostrar suas categorias. Para acionar a consulta de filtro na interação de passar o mouse ou clicar, usaremos o [operador de composição Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o cursor sobre os pontos no gráfico de dispersão abaixo para ver um destaque e uma categoria de título. Clique com a tecla Shift pressionada para tornar as anotações persistentes e visualizar vários rótulos de uma só vez. Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público em geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!_

In [ ]:
hover = alt.selection_single(
    on='mouseover',  # Selecionar o cursor
    nearest=True,    # Seleciona o ponto mais próximo do cursor
    empty='none'     # Seleção vazia não retorna nada
)

click = alt.selection_multi(
    empty='none' # Seleção vazia não retorna nada
)

# scatter plot dos parâmetros citados
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# Camadas
base = plot.transform_filter(
    hover | click # Filtrar os pontos em ambas as direções
)

# Detalhes adcionais da visualização
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

<!--The example above adds three new layers to the scatter plot: a circular annotation, white text to provide a legible background, and black text showing a film title. In addition, this example uses two selections in tandem:

1. A single selection (`hover`) that includes `nearest=True` to automatically select the nearest data point as the mouse moves.
2. A multi selection (`click`) to create persistent selections via shift-click.

Both selections include the set `empty='none'` to indicate that no points should be included if a selection is empty. These selections are then combined into a single filter predicate &mdash; the logical _or_ of `hover` and `click` &mdash; to include points that reside in _either_ selection. We use this predicate to filter the new layers to show annotations and labels for selected points only.-->

O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto preto mostrando o título de um filme. Além disso, este exemplo usa duas seleções em conjunto:

1. Uma única seleção (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo conforme o cursor se move.
2. Uma seleção múltipla (`clique`) para criar seleções persistentes via shift-clique.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se uma seleção estiver vazia. Essas seleções são então combinadas em uma única condição de filtro &mdash; o _ou_ lógico de `hover` e `click` &mdash; para incluir pontos que residem em _qualquer_ seleção. Usamos essa condição para filtrar as novas camadas para mostrar anotações e rótulos apenas para pontos selecionados.

<!--Using selections and layers, we can realize a number of different designs for details on demand! For example, here is a log-scaled time series of technology stock prices, annotated with a guideline and labels for the date nearest the mouse cursor:-->

Usando seleções e camadas, podemos realizar vários designs diferentes para obter detalhes sob demanda! Por exemplo, aqui está uma série temporal em escala logarítmica de preços de ações de tecnologia, anotada com uma diretriz e categorias para a data mais próxima do cursor:

In [ ]:
# selecione um ponto para o qual fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limitar a seleção ao valor do eixo x
    on='mouseover',  # selecione em eventos de ausência de cursor
    nearest=True,    # selecione o ponto de dados mais próximo do cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# definir nosso gráfico de linha base de preços de ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico de linha básica

    # adicionar uma marca para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adicionar marcas circulares para pontos de tempo selecionados, ocultar pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adicionar texto branco para fornecer um fundo legível para rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adicionar categorias de texto para o preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

<!--_Putting into action what we've learned so far: can you modify the movie scatter plot above (the one with the dynamic query over years) to include a `rule` mark that shows the average IMDB (or Rotten Tomatoes) rating for the data contained within the year `interval` selection?_-->

_Colocando em ação o que aprendemos até agora: você pode modificar o gráfico de dispersão do filme acima (aquele com a consulta dinâmica ao longo dos anos) para incluir uma marca de `regra` que mostre a classificação média do IMDB (ou Rotten Tomatoes) para os dados contidos na seleção de `intervalo` do ano?_

<!--## Brushing &amp; Linking, Revisited-->

## Exploração e Vinculação, Revisão

<!--Earlier in this notebook we saw an example of _brushing &amp; linking_: using a dynamic query histogram to highlight points in a movie rating scatter plot. Here, we'll visit some additional examples involving linked selections.

Returning to the `cars` dataset, we can use the `repeat` operator to build a [scatter plot matrix (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) that shows associations between mileage, acceleration, and horsepower. We can define an `interval` selection and include it _within_ our repeated scatter plot specification to enable linked selections among all the plots.

_Click and drag in any of the plots below to perform brushing &amp; linking!_
-->
Anteriormente neste _notebook_ vimos um exemplo de _exploração e vinculação_: usando um histograma de consulta dinâmica para destacar pontos em um gráfico de dispersão de classificação de filme. Agora, visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao conjunto de dados `cars` (carros), podemos usar o operador `repeat` para construir uma [matriz de gráfico de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência. Podemos definir uma seleção de `intervalo` e incluí-la _dentro_ de nossa especificação de gráfico de dispersão repetido para permitir seleções vinculadas entre todos os gráficos.

_Clique e arraste em qualquer uma das visualizações abaixo para realizar a exploração e vinculação!_

In [ ]:
brush = alt.selection_interval(
    resolve='global' # Seleção 'resolve' definida globalmente
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

<!--Note above the use of `resolve='global'` on the `interval` selection. The default setting of `'global'` indicates that across all plots only one brush can be active at a time. However, in some cases we might want to define brushes in multiple plots and combine the results. If we use `resolve='union'`, the selection will be the _union_ of all brushes: if a point resides within any brush it will be selected. Alternatively, if we use `resolve='intersect'`, the selection will consist of the _intersection_ of all brushes: only points that reside within all brushes will be selected.

_Try setting the `resolve` parameter to `'union'` and `'intersect'` and see how it changes the resulting selection logic._-->

Observe acima o uso de `resolve='global'` na seleção de `interval`. A configuração padrão `'global'` indica que em todos os gráficos apenas um pincel pode estar ativo por vez. No entanto, em alguns casos, podemos querer definir pincéis em vários gráficos e combinar os resultados. Se usarmos `resolve='union'`, a seleção será a _união_ de todos os pincéis de entrada: se um ponto estiver dentro de qualquer pincel ele será selecionado. Alternativamente, se usarmos `resolve='intersect'`, a seleção consistirá na _interseção_ de todos os pincéis de entrada: apenas os pontos que estão dentro de todos os pincéis serão selecionados.

_Tente definir o parâmetro `resolve` para `'union'` e `'intersect'` e veja como isso muda a lógica de seleção resultante._